In [1]:
import re
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer 

# Dataframe

In [2]:
df = pd.read_pickle('../data/pickle/podcasts_clean.pkl')

In [3]:
df['text'] = df[['title', 'producer', 'genre', 'description', 'episodes', 'reviews']].agg(' '.join, axis=1)

In [4]:
df.drop(columns=['producer', 'rating', 'num_ratings', 'num_episodes', 'description',
                 'link', 'episodes', 'reviews'], inplace=True)

In [5]:
df['subs_len'] = df.apply(lambda row: len(row.subs), axis=1)

In [6]:
df.head(2)

,title,genre,subs,text,subs_len
0,Green Eggs and Dan,Arts,"[Point of Origin, Cal's Week in Review]",Green Eggs and Dan The Podglomerate Arts Takin...,2
1,Audio Poem of the Day,Arts,"[The New Yorker: Poetry, The New Yorker: The W...",Audio Poem of the Day Poetry Foundation Arts A...,5


# Preprocessing

In [7]:
stopwords = set(nltk.corpus.stopwords.words('english'))
add_stops = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
             'january', 'february', 'march', 'april', 'may', 'june', 'im', 'ive',
             'july', 'august', 'september', 'october', 'november', 'december',
             'nan', 'podcast', 'podcasts', 'every', 'new', 'weekly', 'week', 
             'stories', 'story', 'episode', 'episodes', 'listen', 'us', "'s", 'host', 'hosted', 'join']
for i in add_stops:
    stopwords.add(i)

In [8]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\s\w]+', '', text)
    text = re.sub(r"\S+\.org\S+", "", text)
    text = re.sub(r"\S+\.net\S+", "", text)
    text = re.sub(r"\S+\.edu\S+", "", text)
    text = re.sub(r"\S+\.gov\S+", "", text)
    tokenized_text = nltk.word_tokenize(text)
    new_tokenized = []
    for i in tokenized_text:
        if i not in stopwords and len(i) != 1:
            new_tokenized.append(lemmatizer.lemmatize(i))
    return(' '.join(new_tokenized))

In [9]:
df.text = df.text.map(preprocess_text)

In [10]:
df.head(2)

,title,genre,subs,text,subs_len
0,Green Eggs and Dan,Arts,"[Point of Origin, Cal's Week in Review]",green egg dan podglomerate art taking look eat...,2
1,Audio Poem of the Day,Arts,"[The New Yorker: Poetry, The New Yorker: The W...",audio poem day poetry foundation art audio rec...,5


# Modelling

In [45]:
tests = list(df[df.subs_len != 0].sample(3).title)

In [12]:
def get_recommendations(matrix):
    for i in tests:
        print('\033[1m' + "Given:" + '\033[0m', i)
        index = df.loc[df.title == i].index[0]
        print('\033[1m' + "Given genre:" + '\033[0m', df.iloc[index]['genre'])
        array = list(enumerate(matrix[index]))
        sorted_array = sorted(array, key=lambda x:x[1], reverse=True)
        recs = []
        genres = []
        for j in range(6):
            if j == 0:
                continue
            rec_title = df.iloc[sorted_array[j][0]]['title']
            recs.append(rec_title)
            rec_genre = df.iloc[sorted_array[j][0]]['genre']
            genres.append(rec_genre)
        print('\033[1m' + "Top 5 recommendations:" + '\033[0m')
        print(recs)
        print('\033[1m' + "Top 5 recommendations' genre:" + '\033[0m')
        print(genres)
        print('\033[1m' + "Subscribers also subscribes to according to Apple Podcasts:" + '\033[0m')
        for k in df.loc[df.title == i].subs:
            substo = k
        print(substo)
        correct  = 0
        for l in recs:
            correct = correct + 1 if l in substo else correct
        print('\033[1m', correct , "out of 5 are accurate" + '\033[0m'+ "\n")

## CountVectorizer (Bag-of-words) + Cosine Similarity

In [35]:
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
def CountVectorizer(data):
    bigset = set()
    for sent in data:
        for word in sent.split(' '):
            bigset.add(word)
    vocab = {}
    for index, word in enumerate(sorted(list(bigset))):
        vocab[word] = index
    row, col, val = [],[],[]
    for idx, sentence in enumerate(data):
        count_word = dict(Counter(sentence.split(' ')))
        for word, count in count_word.items():
            col_index = vocab.get(word)
            if col_index >= 0:
                row.append(idx)
                col.append(col_index)
                val.append(count)
    return csr_matrix((val, (row, col)), shape=(len(data), len(vocab)))

In [41]:
bow_matrix = CountVectorizer(df.text)
bow_cos_sim = cosine_similarity(bow_matrix)

In [46]:
get_recommendations(bow_cos_sim)

Given: Joyce Meyer Ministries TV Podcast
Given genre: Religion & Spirituality
Top 5 recommendations:
['Joyce Meyer Enjoying Everyday Life® TV Audio Podcast', 'Joyce Meyer Radio Podcast', "Joyce Meyer's Talk It Out Podcast", "The Potter's House At One LA", 'Equip and Empower with Christine Caine']
Top 5 recommendations' genre:
['Religion & Spirituality', 'Religion & Spirituality', 'Religion & Spirituality', 'Religion & Spirituality', 'Religion & Spirituality']
Subscribers also subscribes to according to Apple Podcasts:
['Joyce Meyer Enjoying Everyday Life® TV Audio Podcast', 'Joyce Meyer Radio Podcast', 'Joel Osteen Podcast', "The Potter's Touch on Lightsource.com", "Pastor Rick's Daily Hope", 'Elevation with Steven Furtick', 'Joel Osteen Podcast', 'Jesus Calling: Stories of Faith', 'Your Move with Andy Stanley Podcast', 'Bethel Church Sermon of the Week', 'The Proverbs 31 Ministries Podcast', '1 Year Daily Audio Bible', 'Transformation Church', 'Craig Groeschel Leadership Podcast']
 2 